In [32]:
from ptk import *

We consider two limit state functions:

$Z_1 = 1.9 - (a+b)$

$Z_2 = 1.85 - (1.5 \cdot b + 0.5 \cdot c)$

Note that both functions have a common variable $b$.

We consider two possible fault trees:
* Failure occurs when both $Z_1$ and $Z_2$ are less than zero
* Failure occurs when either $Z_1$ or $Z_2$ are less than zero

In the first case we want to calculate the probability $P(Z_1<0 \cap Z_2<0>)$ and, in the second case, the probability $P(Z_1<0 \cup Z_2<0)$.

In [33]:
def limit_state_fuction_1(a, b):
    return 1.9 - (a+b)

def limit_state_fuction_2(b, c):
    return 1.85 - (1.5 * b + 0.5 * c)

In [34]:
project = ReliabilityProject()
project.settings.reliability_method = "form"
project.settings.variation_coefficient = 0.02
project.settings.maximum_iterations = 50

project.model = limit_state_fuction_1

project.variables["a"].distribution = "uniform"
project.variables["a"].minimum = -1
project.variables["a"].maximum = 1

project.variables["b"].distribution = "uniform"
project.variables["b"].minimum = -1
project.variables["b"].maximum = 1

project.run()
dp_Z1 = project.design_point

project.model = limit_state_fuction_2

project.variables["c"].distribution = "normal"
project.variables["c"].mean = 0.1
project.variables["c"].deviation = 0.8

project.run()
dp_Z2 = project.design_point

print(f"reliability index Z1 = {dp_Z1.reliability_index}")
print(f"reliability index Z2 = {dp_Z2.reliability_index}")

reliability index Z1 = 2.772351852475011
reliability index Z2 = 1.9500014680701803


In [35]:
combine_project = CombineProject()

combine_project.design_points.append(dp_Z1)
combine_project.design_points.append(dp_Z2)

### Series system

In [41]:
def fault_tree(combine_project, combine_type):

    print(f"{combine_type} system:")
    combine_project.settings.combine_type = combine_type

    combine_algorithms = ["hohenbichler", "importance_sampling", "directional_sampling"]

    for combine_algorithm in combine_algorithms:

        combine_project.settings.combiner_method = combine_algorithm

        combine_project.run()

        dp_correlated = combine_project.design_point
        print(f"{combine_algorithm}: reliability index = {dp_correlated.reliability_index}")

fault_tree(combine_project, "series")
fault_tree(combine_project, "parallel")

series system:
hohenbichler: reliability index = 1.9063961778047613
importance_sampling: reliability index = 1.9090344903514704
directional_sampling: reliability index = 1.9404110047215088
parallel system:
hohenbichler: reliability index = 3.80395450482239
importance_sampling: reliability index = 4.310774972166523
directional_sampling: reliability index = 4.253767038015627


In [37]:
# combine the design points with uncorrelated b values

# set the self correlation of stochast b, default value is 1
combine_project.correlation_matrix['b'] = 0

combine_project.run()

dp_uncorrelated = combine_project.design_point
print('beta combined uncorrelated = ' + str(dp_uncorrelated.reliability_index))


beta combined uncorrelated = 3.94663378591163
